# ベクトル埋め込み、OpenAI、Astra DBを使った哲学

### AstraPy バージョン

このクイックスタートでは、OpenAIのベクトル埋め込みとDataStax [Astra DB](https://docs.datastax.com/en/astra/home/astra.html)をベクトルストアとして使用して、データの永続化を行う「哲学名言検索・生成器」の構築方法を学びます。

このノートブックの基本的なワークフローを以下に示します。有名な哲学者の名言のベクトル埋め込みを評価・保存し、それらを使用して強力な検索エンジンを構築し、その後、新しい名言の生成器まで作成します！

このノートブックは、ベクトル検索の標準的な使用パターンをいくつか例示しており、[Astra DB](https://docs.datastax.com/en/astra/home/astra.html)を使い始めることがいかに簡単かを示しています。

ベクトル検索とテキスト埋め込みを使用して質問応答システムを構築することの背景については、この優れた実践的なノートブックをご確認ください：[Question answering using embeddings](https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb)。

目次：
- セットアップ
- ベクトルコレクションの作成
- OpenAIへの接続
- ベクトルストアへの名言の読み込み
- 使用例1：**名言検索エンジン**
- 使用例2：**名言生成器**
- クリーンアップ

### 仕組み

**インデックス化**

各引用文はOpenAIの`Embedding`を使用してエンベディングベクトルに変換されます。これらは後の検索で使用するためにVector Storeに保存されます。著者名やその他のいくつかの事前計算されたタグを含むメタデータも一緒に保存され、検索のカスタマイズを可能にします。

![1_vector_indexing](https://user-images.githubusercontent.com/14221764/282422016-1d540607-eed4-4240-9c3d-22ee3a3bc90f.png)

**検索**

提供された検索引用文に類似した引用文を見つけるために、後者はその場でエンベディングベクトルに変換され、このベクトルを使用してストアから類似のベクトル、つまり以前にインデックス化された類似の引用文を検索します。検索は追加のメタデータによってオプションで制約することができます（「スピノザによる、これに類似した引用文を見つけて...」など）。

![2_vector_search](https://user-images.githubusercontent.com/14221764/282422033-0a1297c4-63bb-4e04-b120-dfd98dc1a689.png)

ここでの重要なポイントは、「内容が類似した引用文」がベクトル空間では、互いに距離的に近いベクトルに変換されることです。つまり、ベクトル類似性検索は効果的に意味的類似性を実装します。_これがベクトルエンベディングが非常に強力である主要な理由です。_

以下のスケッチはこのアイデアを伝えようとしています。各引用文は、ベクトルに変換されると、空間内の点になります。この場合、OpenAIのエンベディングベクトルは、他の多くのベクトルと同様に_単位長_に正規化されているため、球面上の点となります。そして、この球面は実際には3次元ではなく、1536次元です！

つまり、本質的に、ベクトル空間での類似性検索は、クエリベクトルに最も近いベクトルを返します：

![3_vector_space](https://user-images.githubusercontent.com/14221764/262321363-c8c625c1-8be9-450e-8c68-b1ed518f990d.png)

**生成**

提案（トピックまたは暫定的な引用文）が与えられると、検索ステップが実行され、最初に返された結果（引用文）がLLMプロンプトに入力されます。このプロンプトは、渡された例_と_初期提案に沿って新しいテキストを発明するよう生成モデルに要求します。

![4_quote_generation](https://user-images.githubusercontent.com/14221764/282422050-2e209ff5-07d6-41ac-99ac-f442e090b3bb.png)

## セットアップ

必要な依存関係をインストールしてインポートします：

In [1]:
!pip install --quiet "astrapy>=0.6.0" "openai>=1.0.0" datasets

In [2]:
from getpass import getpass
from collections import Counter

from astrapy.db import AstraDB
import openai
from datasets import load_dataset

### 接続パラメータ

Astraダッシュボードでデータベースの認証情報を取得してください（[詳細](https://docs.datastax.com/en/astra/astra-db-vector/)）。これらの情報はすぐに必要になります。

例の値：

- API Endpoint: `https://01234567-89ab-cdef-0123-456789abcdef-us-east1.apps.astra.datastax.com`
- Token: `AstraCS:6gBhNmsk135...`

In [3]:
ASTRA_DB_API_ENDPOINT = input("Please enter your API Endpoint:")
ASTRA_DB_APPLICATION_TOKEN = getpass("Please enter your Token")

Please enter your API Endpoint: https://4f835778-ec78-42b0-9ae3-29e3cf45b596-us-east1.apps.astra.datastax.com
Please enter your Token ········


### Astra DBクライアントのインスタンス化

In [4]:
astra_db = AstraDB(
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
)

## ベクターコレクションの作成

コレクション名以外に指定する唯一のパラメータは、保存するベクトルの次元数です。その他のパラメータ、特に検索に使用する類似度メトリックは、オプションです。

In [5]:
coll_name = "philosophers_astra_db"
collection = astra_db.create_collection(coll_name, dimension=1536)

## OpenAIに接続する

### シークレットキーの設定

In [6]:
OPENAI_API_KEY = getpass("Please enter your OpenAI API Key: ")

Please enter your OpenAI API Key:  ········


### 埋め込みのテストコール

入力テキストのリストに対して埋め込みベクトルを取得する方法を簡単に確認してみましょう：

In [7]:
client = openai.OpenAI(api_key=OPENAI_API_KEY)
embedding_model_name = "text-embedding-3-small"

result = client.embeddings.create(
    input=[
        "This is a sentence",
        "A second sentence"
    ],
    model=embedding_model_name,
)

_注意: 上記はOpenAI v1.0+の構文です。以前のバージョンを使用している場合、埋め込みを取得するコードは異なって見えます。_

In [8]:
print(f"len(result.data)              = {len(result.data)}")
print(f"result.data[1].embedding      = {str(result.data[1].embedding)[:55]}...")
print(f"len(result.data[1].embedding) = {len(result.data[1].embedding)}")

len(result.data)              = 2
result.data[1].embedding      = [-0.0108176339417696, 0.0013546717818826437, 0.00362232...
len(result.data[1].embedding) = 1536


## ベクターストアに引用文を読み込む

データセットから名言を取得します。_（このデモで使用するために、[このKaggleデータセット](https://www.kaggle.com/datasets/mertbozkurt5/quotes-by-philosophers)のデータを適応・拡張しました。）_

In [9]:
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]

簡単な検査：

In [10]:
print("An example entry:")
print(philo_dataset[16])

An example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


データセットのサイズを確認してください：

In [11]:
author_count = Counter(entry["author"] for entry in philo_dataset)
print(f"Total: {len(philo_dataset)} quotes. By author:")
for author, count in author_count.most_common():
    print(f"    {author:<20}: {count} quotes")

Total: 450 quotes. By author:
    aristotle           : 50 quotes
    schopenhauer        : 50 quotes
    spinoza             : 50 quotes
    hegel               : 50 quotes
    freud               : 50 quotes
    nietzsche           : 50 quotes
    sartre              : 50 quotes
    plato               : 50 quotes
    kant                : 50 quotes


### ベクターコレクションへの書き込み

引用文の埋め込みを計算し、テキスト自体と後で使用するメタデータと一緒にVector Storeに保存します。

速度を最適化し、呼び出し回数を削減するために、OpenAIの埋め込みサービスに対してバッチ呼び出しを実行します。

引用オブジェクトを保存するには、コレクションの`insert_many`メソッドを使用します（バッチごとに1回の呼び出し）。挿入用のドキュメントを準備する際は、適切なフィールド名を選択してください。ただし、埋め込みベクターは固定の特別な`$vector`フィールドでなければならないことに注意してください。

In [12]:
BATCH_SIZE = 20

num_batches = ((len(philo_dataset) + BATCH_SIZE - 1) // BATCH_SIZE)

quotes_list = philo_dataset["quote"]
authors_list = philo_dataset["author"]
tags_list = philo_dataset["tags"]

print("Starting to store entries: ", end="")
for batch_i in range(num_batches):
    b_start = batch_i * BATCH_SIZE
    b_end = (batch_i + 1) * BATCH_SIZE
    # compute the embedding vectors for this batch
    b_emb_results = client.embeddings.create(
        input=quotes_list[b_start : b_end],
        model=embedding_model_name,
    )
    # prepare the documents for insertion
    b_docs = []
    for entry_idx, emb_result in zip(range(b_start, b_end), b_emb_results.data):
        if tags_list[entry_idx]:
            tags = {
                tag: True
                for tag in tags_list[entry_idx].split(";")
            }
        else:
            tags = {}
        b_docs.append({
            "quote": quotes_list[entry_idx],
            "$vector": emb_result.embedding,
            "author": authors_list[entry_idx],
            "tags": tags,
        })
    # write to the vector collection
    collection.insert_many(b_docs)
    print(f"[{len(b_docs)}]", end="")

print("\nFinished storing entries.")

Starting to store entries: [20][20][20][20][20][20][20][20][20][20][20][20][20][20][20][20][20][20][20][20][20][20][10]
Finished storing entries.


## ユースケース1: **引用検索エンジン**

引用検索機能については、まず入力された引用をベクトルに変換し、それを使用してストアにクエリを実行する必要があります（検索呼び出しにオプションのメタデータを処理することに加えて）。

再利用しやすくするために、検索エンジン機能を関数にカプセル化します。その核心となるのは、コレクションの`vector_find`メソッドです：

In [13]:
def find_quote_and_author(query_quote, n, author=None, tags=None):
    query_vector = client.embeddings.create(
        input=[query_quote],
        model=embedding_model_name,
    ).data[0].embedding
    filter_clause = {}
    if author:
        filter_clause["author"] = author
    if tags:
        filter_clause["tags"] = {}
        for tag in tags:
            filter_clause["tags"][tag] = True
    #
    results = collection.vector_find(
        query_vector,
        limit=n,
        filter=filter_clause,
        fields=["quote", "author"]
    )
    return [
        (result["quote"], result["author"])
        for result in results
    ]

### 検索機能をテストする

引用符のみを渡す場合：

In [14]:
find_quote_and_author("We struggle all our life for nothing", 3)

[('Life to the great majority is only a constant struggle for mere existence, with the certainty of losing it at last.',
  'schopenhauer'),
 ('We give up leisure in order that we may have leisure, just as we go to war in order that we may have peace.',
  'aristotle'),
 ('Perhaps the gods are kind to us, by making life more disagreeable as we grow older. In the end death seems less intolerable than the manifold burdens we carry',
  'freud')]

作成者に制限された検索：

In [15]:
find_quote_and_author("We struggle all our life for nothing", 2, author="nietzsche")

[('To live is to suffer, to survive is to find some meaning in the suffering.',
  'nietzsche'),
 ('What makes us heroic?--Confronting simultaneously our supreme suffering and our supreme hope.',
  'nietzsche')]

以前に引用符で保存したタグの中から検索を制限する：

In [16]:
find_quote_and_author("We struggle all our life for nothing", 2, tags=["politics"])

[('He who seeks equality between unequals seeks an absurdity.', 'spinoza'),
 ('The people are that part of the state that does not know what it wants.',
  'hegel')]

### 無関係な結果の除外

ベクトル類似度検索は一般的に、クエリに最も近いベクトルを返します。これは、より良い結果がない場合でも、やや無関係な結果を含む可能性があることを意味します。

この問題を制御するために、クエリと各結果の間の実際の「類似度」を取得し、それに対してカットオフを実装することで、閾値を超える結果を効果的に破棄できます。
この閾値を正しく調整することは簡単な問題ではありません。ここでは、その方法をお見せします。

これがどのように動作するかを理解するために、以下のクエリを試して、引用文と閾値の選択を変更して結果を比較してみてください。類似度は各結果ドキュメントの特別な`$similarity`フィールドとして返されることに注意してください。これは、検索メソッドに`include_similarity = False`を渡さない限り、デフォルトで返されます。

_注記（数学的に興味のある方へ）：この値は、ベクトル間のコサイン差、つまり2つのベクトルのノルムの積で割ったスカラー積の**0と1の間での再スケーリング**です。言い換えると、これは反対方向のベクトルでは0、平行なベクトルでは+1になります。他の類似度測定方法（コサインがデフォルト）については、`AstraDB.create_collection`の`metric`パラメータと[許可される値に関するドキュメント](https://docs.datastax.com/en/astra-serverless/docs/develop/dev-with-json.html#metric-types)を確認してください。_

In [17]:
quote = "Animals are our equals."
# quote = "Be good."
# quote = "This teapot is strange."

metric_threshold = 0.92

quote_vector = client.embeddings.create(
    input=[quote],
    model=embedding_model_name,
).data[0].embedding

results_full = collection.vector_find(
    quote_vector,
    limit=8,
    fields=["quote"]
)
results = [res for res in results_full if res["$similarity"] >= metric_threshold]

print(f"{len(results)} quotes within the threshold:")
for idx, result in enumerate(results):
    print(f"    {idx}. [similarity={result['$similarity']:.3f}] \"{result['quote'][:70]}...\"")

3 quotes within the threshold:
    0. [similarity=0.927] "The assumption that animals are without rights, and the illusion that ..."
    1. [similarity=0.922] "Animals are in possession of themselves; their soul is in possession o..."
    2. [similarity=0.920] "At his best, man is the noblest of all animals; separated from law and..."


## ユースケース2: **引用文ジェネレーター**

このタスクには、OpenAIの別のコンポーネント、つまりVector Storeへのクエリで取得した入力に基づいて見積もりを生成するLLMが必要です。

また、generate-quote LLM完了タスクのために入力されるプロンプトのテンプレートも必要です。

In [18]:
completion_model_name = "gpt-3.5-turbo"

generation_prompt_template = """"Generate a single short philosophical quote on the given topic,
similar in spirit and form to the provided actual example quotes.
Do not exceed 20-30 words in your quote.

REFERENCE TOPIC: "{topic}"

ACTUAL EXAMPLES:
{examples}
"""

検索と同様に、この機能は便利な関数にラップするのが最適です（内部的には検索を使用します）：

In [19]:
def generate_quote(topic, n=2, author=None, tags=None):
    quotes = find_quote_and_author(query_quote=topic, n=n, author=author, tags=tags)
    if quotes:
        prompt = generation_prompt_template.format(
            topic=topic,
            examples="\n".join(f"  - {quote[0]}" for quote in quotes),
        )
        # a little logging:
        print("** quotes found:")
        for q, a in quotes:
            print(f"**    - {q} ({a})")
        print("** end of logging")
        #
        response = client.chat.completions.create(
            model=completion_model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=320,
        )
        return response.choices[0].message.content.replace('"', '').strip()
    else:
        print("** no quotes found.")
        return None

_注意：埋め込み計算の場合と同様に、Chat Completion APIのコードはOpenAI v1.0以前では若干異なります。_

#### 見積もり生成をテストする

単にテキストを渡すだけです（「引用」ですが、実際にはトピックを提案するだけでも構いません。そのベクトル埋め込みは、ベクトル空間内の適切な場所に配置されるためです）：

In [20]:
q_topic = generate_quote("politics and virtue")
print("\nA new generated quote:")
print(q_topic)

** quotes found:
**    - Happiness is the reward of virtue. (aristotle)
**    - Our moral virtues benefit mainly other people; intellectual virtues, on the other hand, benefit primarily ourselves; therefore the former make us universally popular, the latter unpopular. (schopenhauer)
** end of logging

A new generated quote:
True politics lies in the virtuous pursuit of justice, for it is through virtue that we build a better world for all.


単一の哲学者からのインスピレーションを使用してください：

In [21]:
q_topic = generate_quote("animals", author="schopenhauer")
print("\nA new generated quote:")
print(q_topic)

** quotes found:
**    - Because Christian morality leaves animals out of account, they are at once outlawed in philosophical morals; they are mere 'things,' mere means to any ends whatsoever. They can therefore be used for vivisection, hunting, coursing, bullfights, and horse racing, and can be whipped to death as they struggle along with heavy carts of stone. Shame on such a morality that is worthy of pariahs, and that fails to recognize the eternal essence that exists in every living thing, and shines forth with inscrutable significance from all eyes that see the sun! (schopenhauer)
**    - The assumption that animals are without rights, and the illusion that our treatment of them has no moral significance, is a positively outrageous example of Western crudity and barbarity. Universal compassion is the only guarantee of morality. (schopenhauer)
** end of logging

A new generated quote:
Excluding animals from ethical consideration reveals a moral blindness that allows for their explo

## クリーンアップ

このデモで使用したすべてのリソースを削除したい場合は、このセルを実行してください（_警告: これによりコレクションとそのデータが不可逆的に削除されます！_）：

In [22]:
astra_db.delete_collection(coll_name)

{'status': {'ok': 1}}